# Performance by product --> SoM Dashboard

In [1]:
CPG = 'rb'

## Importar

In [2]:
# %load basic
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',200)
pd.options.display.float_format = '{:.2f}'.format
liga = '/Users/efraflores/Desktop/EF/Corner/Catalogo/data/'
sorted(os.listdir(liga))

['.DS_Store',
 'nielsen_coca.csv',
 'nielsen_coca_v2.csv',
 'nielsen_danone.csv',
 'nielsen_essity.csv',
 'nielsen_rb.csv',
 'pbbc.csv',
 'perf_prod_2005_2103.csv',
 'products_coca.csv',
 'products_coca_correct.csv',
 'products_danone.csv',
 'products_danone_may20_mar21.csv',
 'products_essity.csv',
 'products_essity_correct.csv',
 'products_rb.csv',
 'products_rb_correct.csv',
 'prov_coca.csv',
 'prov_essity.csv',
 'prov_rb.csv',
 'som_coca.csv',
 'som_danone.csv',
 'som_essity.csv',
 'som_rb.csv']

In [3]:
import chardet
with open(os.path.join(liga,f'products_{CPG.lower()}.csv'), 'rb') as rawdata:
    encod = chardet.detect(rawdata.read(1000))['encoding']
print(encod)

UTF-16


In [4]:
df = pd.read_csv(liga + f'products_{CPG.lower()}.csv',
                 sep='\t', 
                 encoding=encod
                )
df.sample(2)

/Users/efraflores/Desktop/corner/corner/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (10,11,12,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Mes de date,store,category,category_id,product ID,product name,product barcodes,product package,brand name,audience,orders,sales,users,avg. ticket,units found,units requested,found rate
235472,abril 2021,Fresh Market,Aseo de ba√±o y cocina,988,336393,Desinfectante antibacterial crisp linen,{7501058752796},475 g,Lysol,non pop,38,7244,NaN,185.77,58,58,100.0
81739,septiembre 2020,Chedraui,Laundry,984,336586,Detergente líquido para ropa limpieza fresca,{7501080954724},6.4 l,Arm & Hammer,pop,238,"40,210",205,169,243,267,91.01%


In [5]:
nielsen = pd.read_csv(liga + f'nielsen_{CPG.lower()}.csv').drop_duplicates()
nielsen['UPC'] = nielsen['UPC'].astype(int,errors='ignore')
nielsen.sample(2)

,UPC,FORMATO,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
3530,NaN,NaN,NaN,NaN,NaN,Elet,4 Lt,Lavaplatos Mandarina,Laundry,NaN,NaN,NaN
5192,7501071904004,NaN,Clorox MX,NaN,NaN,Green Works,500 ml,Limpiador multiusos con ingredientes naturales,Bathroom and kitchen cleaning,NaN,NaN,NaN


## Funciones

In [6]:
import time
import numpy as np
from IPython.lib.display import Audio

start = time.time()
def time_exp(x):
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

In [7]:
def limpio(data,mes_col):
    data[['mes','anio']] = data[mes_col].str.split(expand=True)
    meses = dict(zip(['enero','febrero','marzo','abril','mayo','junio','julio','agosto','septiembre','octubre','noviembre','diciembre'],range(1,13)))
    #Si la data está en inglés, activar!
    #data['anio'] = '20' + data['anio'].astype(str)
    #meses = dict(zip(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'],range(1,13)))
    data['mes'] = data['mes'].replace(meses)
    def titles(text):
        return str(text).replace('.','').replace(' ','_').lower()
    data.columns = [titles(x) for x in data.columns]
    return data.drop(columns = [titles(mes_col),'avg_ticket','found_rate'])

## Transformar

In [8]:
df['product barcodes'] = df['product barcodes'].str.replace('{','').str.replace('}','').str.split(',',n=1).fillna(".").apply(lambda x:x[0])

<ipython-input-8-8849c05c0c2f>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['product barcodes'] = df['product barcodes'].str.replace('{','').str.replace('}','').str.split(',',n=1).fillna(".").apply(lambda x:x[0])


In [9]:
df = limpio(df.merge(nielsen.fillna({'UPC':'0'}).astype(int,errors='ignore').astype(str),
                     left_on='product barcodes',
                     right_on='UPC'),'Mes de date')
df.sample(2)

,store,category,category_id,product_id,product_name,product_barcodes,product_package,brand_name,audience,orders,sales,users,units_found,units_requested,upc,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3,mes,anio
16769,Chedraui,Bathroom and kitchen cleaning,988,336121,Cloro max control,7501025405212,950 ml,Cloralex,non pop,257,"7,461",242,339,355,7501025405212,nan,Alen,nan,nan,Cloralex,950 ml,Cloro max control,Bathroom and kitchen cleaning,nan,nan,nan,10,2020
219072,City Market,Bathroom and kitchen cleaning,988,315119,Limpiador de inodoro fresco,7501071905865,709 ml,Clorox,pop,3,192,3,4,4,7501071905865,nan,Clorox MX,nan,nan,Clorox,709 ml,Limpiador de inodoro fresco,Bathroom and kitchen cleaning,nan,nan,nan,2,2020


## Exportar

In [10]:
df.to_csv(os.path.join(liga,f'som_{CPG.lower()}.csv'),index=False,sep='\t',encoding='utf-16')

## Fin

In [11]:
time_exp(time.time() - start)
tono()

0 minutos con 4.25 segundos
